## Emaple of Reachability Problem of Quadratizition of ODEs

---

### 1. First Example

$$
\begin{cases}
y' = -2y+2ay^{2}\\
x' = -x+axy
\end{cases}
$$
where we have $y=x^2$

In [ ]:
# =================
# Dependencies
# =================

using CarlemanLinearization
using CarlemanLinearization: kron_sum,
    error_bound_pseries,
    error_bound_specabs,
    convergence_radius_specabs,
    _error_bound_specabs_R,
    build_matrix

using ReachabilityAnalysis, CarlemanLinearization
using Plots, LaTeXStrings, LinearAlgebra, SparseArrays
using Plots.PlotMeasures

using LazySets: center
using CarlemanLinearization: _error_bound_specabs_R

include("../utils.jl")

In [12]:
# =================
# Model definition
# =================

a = 1
function system_carlin()

  F1 = zeros(2, 2)
  F1[1, 1] = -1
  F1[2, 2] = -2

  F2 = zeros(2, 4) # [x, x⊗x]
  F2[1, 4] = 2 * a
  F2[2, 2] = a

  return F1, F2
end

function _solve_system_carlin(; N=4, T=30.0, δ=0.1, radius0=0, bloat=false, resets=nothing)
  x0c = [0.0, 0.0]

  F1, F2 = system_carlin()
  R, Re_lambda1 = _error_bound_specabs_R(x0c, F1, F2; check=true)

  n = 2
  dirs = _template(n=n, N=N)
  print("dirs: ", dirs)
  alg = LGG09(δ=δ, template=dirs)

  if radius0 == 0
    X0 = convert(Hyperrectangle, Singleton(x0c))
  else
    X0 = Hyperrectangle(x0c, radius0)
  end

  if isnothing(resets)
    @time sol = _solve_CARLIN(X0, F1, F2; alg=alg, N=N, T=T, bloat=bloat)
  else
    @time sol = _solve_CARLIN_resets(X0, F1, F2; resets=resets, alg=alg, N=N, T=T, bloat=bloat)
  end

  return sol

end

_solve_system_carlin(; N=2, T=1.0, δ=0.1, radius0=0, bloat=false, resets=nothing)


dirs: CustomDirections{Float64, SingleEntryVector{Float64}}(SingleEntryVector{Float64}[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, -1.0, 0.0, 0.0, 0.0, 0.0]], 6, false, true) 10.536160 seconds (11.64 M allocations: 579.495 MiB, 19.02% gc time, 99.59% compilation time: 1% of which was recompilation)


Flowpipe{Float64, ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}, Vector{ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}}}(ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}[ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}(HPolyhedron{Float64, Vector{Float64}}(HalfSpace{Float64, Vector{Float64}}[HalfSpace{Float64, Vector{Float64}}([1.0, 0.0], 0.0), HalfSpace{Float64, Vector{Float64}}([0.0, 1.0], 0.0), HalfSpace{Float64, Vector{Float64}}([-1.0, 0.0], 0.0), HalfSpace{Float64, Vector{Float64}}([0.0, -1.0], 0.0)]), [0, 0.100001]), ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}(HPolyhedron{Float64, Vector{Float64}}(HalfSpace{Float64, Vector{Float64}}[HalfSpace{Float64, Vector{Float64}}([1.0, 0.0], 0.0), HalfSpace{Float64, Vector{Float64}}([0.0, 1.0], 0.0), HalfSpace{Float64, Vector{Float64}}([-1.0, 0.0], 0.0), HalfSpace{Float64, Vector{Float64}}([0.0, -1.0], 0.0)]), [0.1, 0.200001]), ReachSet{Float64, HPolyhedron{Float64, Vector{Float64}}}(HPolyhed

In [41]:
@taylorize function system_equation(dx, x, a)
  x, y = x

  dx[1] = -2*y+2*a*y^2
  dx[2] = -x+a*x*y
end

In [42]:
function _solve_system_carlin_TM(; T=30.0, radius0=0, trajectories=-1)
  x0c = [0.0, 0.0]

  if radius0 == 0
    X0 = convert(Hyperrectangle, Singleton(x0c))
  else
    X0 = Hyperrectangle(x0c, radius0)
  end

  @time sol = solve(
    X0,
    tspan=(0.0, T),
    alg=TMJets(δ=0.1),
    orderT=10,
    orderQ=2,
    trajectories=trajectories,
    vars=(0, 1),
    a=a,
  )

  println(sol)
  return sol
end

_solve_system_carlin_TM (generic function with 1 method)

---

### 2. Plot of the first example

In [45]:
# ===============
# Results
# ===============

# parameters
Tmax = 30.0
rr0 = 0.0

# # taylor models solution
# sol_tm = _solve_system_carlin_TM(T=Tmax, radius0=rr0)
# time_TM = @elapsed _solve_system_carlin_TM(T=Tmax, radius0=rr0)

# no error bounds, N = 2
_solve_system_carlin(N=2, T=Tmax, δ=0.1, radius0=rr0, bloat=false)
time_NoError_N2 = @elapsed _solve_system_carlin(N=2, T=Tmax, δ=0.1, radius0=rr0, bloat=false)

print(io, "SEIR, Carleman, no error bound, N=2, $(time_NoError_N2)\n")

LoadError: AssertionError: the problems' dimension 6 doesn't match the dimension of the template directions, 12